# NBA 2014-2015 Season Player Data

## [Time to play Moneyball](https://www.youtube.com/watch?v=yGf6LNWY9AI)

Let's try all the things we've learned on a dataset from start to end. I've grabbed end of year stats for the 2014-2015 season for the players in the NBA.

The dataset originally comes from [Basketball Reference](http://www.basketball-reference.com/leagues/NBA_2015_totals.html?lid=header_seasons#totals::none). There's been nearly 0 preprocessing done, to simulate what would happen in the real world.

**Column Names:**

```
    Rk: Rank (No meaning)
    Player: Player name
    Pos: Position (PF - power forward, SG - shooting guard, C - Center,  SF - small forward, PG - point guard)
    Age: age
    Tm: Team
    G: Number games played
    GS: Games started
    MP: Minutes Played
    FG: Field goals Made
    FGA: Field goal attempts
   *FG%: Field goal percentage
    3P: 3 Point field goals made
    3PA: 3 Point field goal attempts
   *3P%: 3 Point field goal percentage
    2P: 2 Point field goals made
    2PA: 2 Point field goal attempts
    2P%:  2 Point field goal percentage
    eFG%: Effective Field goal percentage, takes 3 vs 2 point into account
    FT: Free throws made
    FTA: Free throw attempts
   *FT%: Free throw percentage
    ORB: Offensive rebounds
    DRB: Defensive rebounds
   *TRB: Total Rebounds
   *AST: Assists
   *STL: Steals
   *BLK: Blocks
   *TOV: Turnovers
    PF: Personal Fouls
   *PTS: points
```
    
We'll go over:
* Loading the data
* Preprocessing the data
* Looking at some basic stats
* Plotting some of these stats for intuition
* Running K Nearest Neighbors on the data


In [ ]:
# Import the packages we are going to use
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Read in the dataset from the repo (Update your repo if you don't see this dataset)
bb = pd.read_csv('leagues_NBA_2015_totals_totals.csv')

In [ ]:
# As always, first thing is to look at the head
bb.head()

In [ ]:
# What are all the columns?
bb.columns

In [ ]:
# Set the max number of columns to display
pd.set_option('display.max_columns', 35)
bb.head()

In [ ]:
# Great, so we know there are 675 rows and 30 columns
# In addition, there seem to be nulls in some of the columns (i.e FG%)
bb.info()

In [ ]:
# Let's run our descriptive tool...
bb.describe()

What the heck? The top occuring "Player" is "Player"? What is going on...

In [ ]:
bb['Player'].value_counts() #Let's look at the player column only

In [ ]:
# Hmm.. lets grab a few rows where the Player name is "Player"
bb[ bb['Player'] == 'Player' ].head()

In [ ]:
#Another way to do look at the rows where Player name is "Player'
bb.iloc[ np.where(bb['Player'] == 'Player') ].head()

I see. It seems the columns were included multiple times in the dataset. Let's go ahead and delete them.

In [ ]:
# bb = bb.drop(  bb[ bb['Player'] == 'Player' ].index ) #One way, using 'drop'
bb = bb[ bb.Player != 'Player' ] #Cleaner way

In [ ]:
bb.describe()

In [ ]:
# Let's choose rows where player is Tayshaun Prince
bb[ bb.Player == 'Tayshaun Prince' ]

Ok, so this guy's been traded around a bit.  But it doesn't make sense.  What the heck team is "TOT" ?
After looking at this guy's wikipedia, I see he's played for Memphis, Boston, and Detroit. Also, if you add up the number of games he's played for MEM + BOS + DET, it comes out to 58 exactly. So then, there isn't really a "TOT" team, its really just the total stats for that year for him.

In [ ]:
# So how many times have players been under a 'TOT' team (i.e. how many players have been traded mid year)
# If its a low number, then maybe get rid of entries with team == TOT
bb.Tm.value_counts()

In [ ]:
# Oh, thats quite a lot of players. Lets sort them by #Points to see if this is important or not
bb[ bb['Tm'] == 'TOT' ].sort('PTS').head()

Wait. what? We sorted by PTS, but the order seems wrong... What is the type of PTS?

In [ ]:
bb.PTS.head()

Okay, so it seems these integers got string representations. Let's check the values to see if there's any crazy values that keep us from converting to ints

In [ ]:
bb.PTS.values

In [ ]:
bb.PTS = np.int64( bb.PTS )

In [ ]:
# Ok, lets' retry sorting by points. Let's do descending now, since that makes more sense
bb[ bb['Tm'] == 'TOT' ].sort('PTS',ascending=False).head()

In [ ]:
#Lets see what the data looks like now
bb.describe()

What the heck? Ah, its running describe() on the numerical columns. Let's go ahead and force dtypes on these columns

In [ ]:
bb.columns

In [ ]:
intCols = ['Age','G','GS','MP','FG','FGA','3P','3PA',
           '2P','2PA','FT','FTA','ORB','DRB','TRB',
           'AST','STL','BLK','TOV','PF','PTS']

for col in intCols:
    bb[col] = np.int64( bb[col] )
        

In [ ]:
fltCols = ['FG%','3P.1','2P.1','eFG%','FT%',]
for col in fltCols:
    bb[col] = np.float64( bb[col] )

In [ ]:
#lets run describe again, see if this makes sense
bb.describe()

In [ ]:
#Lets see if for instance, our float conversion makes sense. Does the average player have roughly 50% fg?
bb.hist('FG%',bins=20)

In [ ]:
#Ah, what about free throw percentages?
bb.hist('FT%',bins=20) 

In [ ]:
# Let's also dump the "Rk" column since its meaningless
bb = bb.drop('Rk',axis=1)

Alright, we've gotten this column type nonsense out of the way. Let's go back to our original issue of players having been under multiple teams

In [ ]:
#Print out the example of Tayshaun Prince to help guide us in the right way
bb[ bb.Player == 'Tayshaun Prince' ]

**What can we do about handling players under multiple teams? At the end of the day, we just want one row per player.** 

**Possible Options:**

1. Only keep the row that the player ended the season at. Delete the others. (Don't have this data though!)
2. Only keep the row that the player played the most games on. Delete TOT and the other rows
3. Only keep TOT row. Delete the other rows. Keep the team name.
4. Only keep TOT row. Delete the other rows. Rename "TOT" to a different team name.


Our new rule will be, For any player that has been under 'TOT', we will do a few things:
* Keep the TOT row, since we want to get a player's entire stats.
* Rename the team name to the team name under which he's played the most games
* Delete the other rows.
* Is this the best way? Maybe, maybe not, but it seems reasonable

In [ ]:
# First, lets sort to make it more clear
bb = bb.sort('Player')
bb.head(10)

1. Okay. So our game plan is this. For each duplicated player, we're going to grab the row indexes where he exists.

2. Then we will look for the indices where his team is not TOT, and he's played the max games in. 

3. Next, we'll delete the rows for where Tm does not equal TOT. 

4. Finally, we'll rename the Tm to the max_game_team.

In [ ]:
# This of course requires that we have UNIQUE indexes! Let's check if theres any duplicated index labels
np.any(bb.index.duplicated())

In [ ]:
# Find out the unique names of the players with multiple rows
whichDup = bb['Player'].duplicated()
multPlayers = bb['Player'][whichDup].unique()
multPlayers

In [ ]:
#Sample code of what I did to walk through deleting and renaming for a single instance

player = multPlayers[0]
print player, '\n'
print bb[ bb.Player == player ]
nonTotIdx = bb[ (bb.Player == player) & (bb.Tm != 'TOT') ].index
totIdx    = bb[ (bb.Player == player) & (bb.Tm == 'TOT') ].index
print nonTotIdx, totIdx

maxGameTm = bb['Tm'][bb.ix[ nonTotIdx]['G'].idxmax()]
print maxGameTm

# The following would be to assign the team name to TOT, and drop the other rows
#bb.ix[ totIdx,'Tm' ] = maxGameTm
#bb = bb.drop(nonTotIdx,axis=0)


In [ ]:
#Real code we will use to automate this process
for player in multPlayers:
    nonTotIdx = bb[ (bb.Player == player) & (bb.Tm != 'TOT') ].index
    totIdx    = bb[ (bb.Player == player) & (bb.Tm == 'TOT') ].index
    maxGameTm = bb['Tm'][ bb.ix[ nonTotIdx]['G'].idxmax() ]
    bb.ix[ totIdx,'Tm' ] = maxGameTm
    bb = bb.drop(nonTotIdx,axis=0)

bb.head()

--------
# Assist to Turnover Metric Creation

Alright, I think we're starting to get somewhere. The data looks roughly cleaned. Let's get some interesting metrics here. There's a metric of **`assists to turnovers`**.

This measurement is especially effective at measuring ball control, because a player who distributes the ball and gains assists without turning the ball over will have a *high ratio*. The opposite is true of players who turn the ball over more than they distribute it to their teammates for successful scoring opportunities.

Let's calculate that where:
```
ATT = AST / TOV
```

## Exercise

IF you have not already, run the code blocks up until this point.

**1) Calculate the ATT (Assist to turnover) Score and set it to a column of bb. ATT = AST / TOV.**

**2) Great. Let's take a look at who's got some great ball handling skills. Subset bb on the following columns: 'Player','AST','TOV','ATT', and sort the resulting dataframe by ATT descending**

Call .sort(??) on the dataframe after you have subsetted it.

**3) There's some slight issues in our calc. If they have never turned over the ball, their ratio is infinite**

**Question:** If their ratio is infinity, are they the greatest ankle breakers in history...or just noobs and haven't played many games?

**Answer:** They are Noobs. We should replace those values with np.nan

You can replace values such as np.inf in a specific column through the replace function

bb['SomeColumn'] = bb['SomeColumn'].replace( ??? )

**Go ahead and replace all inf or -inf values in bb['ATT'] with np.nan**

**4) Again, let's take a look at who's got some great ball handling skills. Subset bb on the following columns: 'Player','AST','TOV','ATT', and sort the resulting dataframe by ATT descending. Ensure that there are no nans or infs in the printed out dataframe**

Ok, this is better, but we still get some rnaomd guys like Gal Mekel. How should we handle the data so that ATT is calculated over 'valid' players (i.e. guys that have played enough, or have enough points, etc.) What is the limit? At what point should we start considering a player for having a valid AST/TOV ratio? 

**Possible Options**
* Maybe we can compare the AST/TOV ratio to how many games they've played
* Get a quantile of games they've played. Subset on players playing over xx
* *Choose a hard cutoff of minimum assists* **<-- Let's choose 200 as the minimum cutoff for assists**
* Choose a quantile cutoff of minimum assists

http://www.sportingcharts.com/nba/stats/player-assists-to-turnover-ratio/2014/

**5) For any player who has less than 200 assists in the season, set their ATT to np.nan. Repeat question 4, and compare against the stats on the website**

**6) Create a histogram of ATT using 10 bins. Set the xlabel to 'ATT' using plt.xlabel, and the ylabel to 'Count' using plt.ylabel. Set the title as 'ATT Histogram' using plt.title**

Note: You can either create histograms directly from Pandas dataframes, or using plt.histogram and pass in the dataframe

End of Exercise

------

---
# Metric Creation - Eddy Curry Line
The **Eddy Curry Line** was originally established to put a spotlight on how truly anemic Curry's overall fantasy line is. The standard: a player must average more turnovers than assists, steals, and blocks combined – in order to qualify, a player must have appeared in at least half of his team's games and averaged at least 25 minutes of playing time.

```
ECL (Eddy Curry Line) = (Assists + Steals + Blocks) / Turnovers

where:
* ECL is only calculated for players averaging 25 minutes PER game
* We will ignore the team condition for now
* If a player has less than 25 minutes PER game, set his ECL to nan
```

Let's calculate the Eddy Curry Line for everyone. 

In [ ]:
bb['ECL'] = bb[ ['AST','STL','BLK'] ].sum(axis=1) / bb.TOV
bb.ix[ (bb.MP / bb.G) < 25, 'ECL'] = np.nan

In [ ]:
bb[['Player','TOV','AST','STL','BLK','ECL']].sort('ECL',ascending=False).head()

In [ ]:
bb['ECL'].hist(bins=20)

In [ ]:
# Let's see who has ECL less than 1 (Has more turnovers than assists, steals, blocks combined)
bb[ bb.ECL < 1 ]

According to professional roommates, Enes Kanter is "a bad player." That said, Utah traded him to OKC as part of a $70 million deal. I guess OKC didn't check his ECL.

Let's see some more low ECL's

In [ ]:
bb[ bb.ECL < 1.5 ]

Huh, all centers. Is there a pattern here?

In [ ]:
grouped = bb.groupby('Pos')
grouped['ECL'].agg( ['mean','std','min','max','count'] )

#If you wanted these agg stats on all of the columns within each group
#grouped.agg( ['mean','std','min','max','count'] ) 

In [ ]:
# Hmm lets see how many players are actually in these mix positions
bb.Pos.value_counts()

#Not that many, lets see who these players are
bb[ ~bb.Pos.isin(['C','PF','PG','SF','SG'] )]

In [ ]:
# We could have performed a process similar to above (when choosing the team other than TOT)
# But for now, lets just only subset to dataframe to playeres in the main 5 positions
bb = bb[ bb.Pos.isin(['C','PF','PG','SF','SG']) ]

In [ ]:
# Let's take a look at the core positions
grouped = bb.groupby('Pos')
grouped['ECL'].agg( ['mean','std','min','max','count'] )

Looking at a bunch of numbers doesnt help me too much. How about a box plot? 


<img src="http://i.stack.imgur.com/ZN8N6.png" style=""/ align=left>


In [ ]:
# Much more clear!
f = bb.boxplot('ECL',by='Pos')

This tells us something! It's harder to get a good Center than it is to get a good Small Forward or Shooting Guard.

Why? There's such a small spread in Eddy Cury Lines for small forwards or shooting guards, whereas centers have a much larger spread. i.e. General Managers spend a good amount of attention looking for good centers


<img src="https://espnfivethirtyeight.files.wordpress.com/2015/06/paine-bradshaw-nba-draft-model.png?w=610&h=1189" style=""/ align=left>


-------
# 3 Point Attempt Statistics
Let's take a look at some more stats. Which teams and positions are shooting the most threes?

To do that, let's use groupby's in Pandas

In [ ]:
bb.head()

bb_tmp = bb.groupby(['Tm','Pos'])['3PA'].sum().unstack()
bb_tmp.head()

Hmm...numbers look reasonable, but not really easy to visualize. Luckily! There's a heatmap in the `seaborn` package.

If you'd like to install seaborn (it's a visualization package built on top of matplotlib), you can go to your terminal, and type in `pip install seaborn`, and restart python.

In [ ]:
import seaborn as sb

In [ ]:
plt.figure(figsize=(15,15)) #This line helps resize the figure to be larger
f = sb.heatmap(bb_tmp,annot=True,fmt="f",linewidths=.5)

Sweet! The general trend makes sense. PG and SG positions throw a lot of threes. Power forwards and Centers attempt less three point shots. Notice anything with the heatmap values for centers? What is going on? Let's take a look at the actual rows.

In [ ]:
bb[ (bb.Tm=='MIA') & (bb.Pos=='C')]

Ah makes sense now! Chris Bosh plays center position, but can shoot threes.

------------
# Metric Creation - 3 Point Efficiency

We've looked at 3 Point attempts. 

3 Point Efficiency is also a heavily utilized metric (i.e. 3 Pt shot attemps vs. 3 Pt Shot %)

Comparing the shot volume vs. shot % gives us an indication of who's a shot chuckers and who's a shot caller.

Let's generate a scatter of 3PA vs 3P.1 (3 point attempts vs. 3 point %)

In [ ]:
f = plt.scatter( bb['3PA'], bb['3P.1'],alpha=.5)

Unfortunately, it's hard to get a good idea of if theres any shape to this data. Too many guys have made 0 or 1 attempts, have have some efficiency. Let's try to filter out guys who have attempted less than X shots.

In [ ]:
shot_attempt_q = bb['3PA'].quantile( q = np.linspace(0,1,11) )
shot_attempt_q

In [ ]:
# Going to subset on players who have attempted at least 28.2 shots (roughly 60% of players have shot more than 28.2 3 pointers)

bb_tmp = bb[ bb['3PA'] > shot_attempt_q[0.4] ]
plt.scatter( bb_tmp['3PA'], bb_tmp['3P.1'],alpha=.5)

Hm, I wonder if certain positions are more efficient 3 point shooters? What if we gave a single color dot to each player type in the scatter?

Let's get some colors in here to help visualize better. 

1. Let's see how many colors we'll need (i.e. how many types of positions are there?
2. Let's define a color for each position. [Rainbow of colors](http://stackoverflow.com/questions/22408237/named-colors-in-matplotlib)
3. Then, we'll create a mapping between 'Position' and 'color' name
4. Finally, for each player position, we'll create a scatter, using the specific color for that position

In [ ]:
# We've already gotten rid of the 'junk' multiple-position rows
bb.Pos.value_counts() 

In [ ]:
# So many colors to choose from! http://stackoverflow.com/questions/22408237/named-colors-in-matplotlib
colors = pd.Series( ['r','g','b','y','c'], index=['SG','PG','SF','PF','C'])
colors

In [ ]:
shot_attempt_q = bb['3PA'].quantile( q = np.linspace(0,1,11) )

for pos in bb.Pos.unique():
    bb_tmp = bb[ (bb['3PA'] > shot_attempt_q[0.4]) & (bb.Pos == pos) ]
    plt.scatter( bb_tmp['3PA'], bb_tmp['3P.1'], c = colors[pos],label=pos,s=50,alpha=.5)
    plt.xlabel('3 Point Attempted'), plt.ylabel('3 Point %')

f = plt.legend(bbox_to_anchor=(0, 1), bbox_transform=plt.gcf().transFigure)

In [ ]:
#Let's see who some of these top guys are
bb[ (bb['3PA'] > 400) & (bb['3P.1'] > .4) ] 

In [ ]:
# Let's make our plot prettier by adding labels
# Requires the labels, as well as scatter positions 
labeldf = bb[ (bb['3PA'] > 400) & (bb['3P.1'] > .4) ]
labels, x, y, labelcolors = labeldf['Player'], labeldf['3PA'], labeldf['3P.1'], colors[ labeldf['Pos'] ]

In [ ]:
#Lets annotate a single player having 3PA > 400 and 3P% > 40%
bb_tmp = bb[ bb['3PA'] > shot_attempt_q[0.4] ]


for pos in bb_tmp.Pos.unique():
    bb_tmp2 = bb_tmp[ bb_tmp.Pos == pos ]
    plt.scatter( bb_tmp2['3PA'], bb_tmp2['3P.1'], c = colors[pos],label=pos,s=50,alpha=.5)
    plt.xlabel('3 Point Attempted'), plt.ylabel('3 Point %')

f = plt.legend(bbox_to_anchor=(0, 1), bbox_transform=plt.gcf().transFigure)

plt.annotate(labels.iloc[0], 
             xy = (x.iloc[0],y.iloc[0]), 
             xytext=(-20,20), 
             textcoords='offset points', 
             arrowprops={'arrowstyle': '->', 'connectionstyle':'arc3,rad=0'})

In [ ]:
#Lets do this for the first couple players now
bb_tmp = bb[ bb['3PA'] > shot_attempt_q[0.4] ]


for pos in bb_tmp.Pos.unique():
    bb_tmp2 = bb_tmp[ bb_tmp.Pos == pos ]
    plt.scatter( bb_tmp2['3PA'], bb_tmp2['3P.1'], c = colors[pos],label=pos,s=50,alpha=.5)
    plt.xlabel('3 Point Attempted'), plt.ylabel('3 Point %')

f = plt.legend(bbox_to_anchor=(0, 1), bbox_transform=plt.gcf().transFigure)

for i in range(len(labels)):
    plt.annotate(labels.iloc[i], 
             xy = (x.iloc[i],y.iloc[i]), 
             xytext=(20,-20), 
             textcoords='offset points', 
             arrowprops={'arrowstyle': '->', 'connectionstyle':'arc3,rad=0'})

# Exercise: There's so much more!

MatPlotLib PyPlot alone has many different types of Plots that you can show.

[MatPlotLib Gallery of plots](http://matplotlib.org/gallery.html)


Create a single new chart/plot/scatter/barplot/boxplot/etc. of interest using the dataset. This can be through the Pandas groupby functions or a PyPlot. For more help on Pandas groupby, see lab 02 on Pandas.

If you would like, also install Seaborn, a library built on top of Matplotlib. The plots tend to look much nicer automatically when you import seaborn. In addition, seaborn has additional plots like the heatmap we just saw:

http://stanford.edu/~mwaskom/software/seaborn/

To install seaborn, go to your terminal and type
```pip install seaborn```
and restart Python

**Ideas:**
* You could compare offense to defense according to some metric (i.e. ORB / DRB). Is this different across positions?
* Do certain position types have more total rebounds?
* Common intuition is that centers have worse shooting percentages than shooting guards. Is this true?


At the end of the exercise, you will have a chance to show your class your plot. Of course, if you're a stats head and have lots of ideas, please share them with the class!


In [ ]:
#Your code here




# KNN on the NBA dataset

We've made some interesting observations so far. For instance, 

* There's a lot more variability in performance in Centers than there is for SG or SF or PG. This implies that if we were a general manager, we might want to pay more to get a good center, than we would to get a good SG or SF. 
* We've also seen that generally the most efficient 3 point shooters are, as expected, shooting guards (The exception being Steph Curry, who was essentially born on a basketball court.) 
* We've ALSO seen that we cannot necessarily always believe the stats, ala Demarcus Cousins who has a low ATT value, but is quite good for a center.

Now that we've gone through this data, let's try to use the model we've just learned, K Nearest Neighbors.

We will try to predict a player's **Position** based on attributes such as FG, FGA, 3P, 3PA, FT, FTA, ORB, DRB, AST, STL, BLK, TOV, PTS, and our newly created items: ATT (assist to turnover) and ECL (Eddy Curry Line). 

In [ ]:
bb.head()


In [ ]:
# First, lets grab only the relevant data columns
bb_knn = bb[ ['Pos','FG','FGA','3P','3PA','FT','FTA','ORB','DRB','AST','STL','BLK','TOV','PTS']]
bb_knn.head()

In [ ]:
# Define the X, and y
X = bb_knn.iloc[:,1:]
y = bb_knn.iloc[:,0]

In [ ]:
# Import the neighbors module from sklearn
from sklearn import neighbors

# In addition, import the train_test_split module to split up our data
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8)  # Use 80% of the data for training

In [ ]:
# Create our model based on 5 neighbors
model = neighbors.KNeighborsClassifier(n_neighbors=5)  # init kNN, with k=5

In [ ]:
# Fit our model using training data
model.fit(X_train, y_train)

In [ ]:
# Test/Score the model on training data
model.score( X_train, y_train )

In [ ]:
# Test/Score the model on testing data
model.score( X_test, y_test )

Let's see what a 5 fold CV score mean looks like

In [ ]:
from sklearn.cross_validation import cross_val_score
cv_scores = cross_val_score(model, X, y, cv=5)  # use 5 different test sets
cv_scores.mean()

## Exercise:

**Build a K Nearest neighbors model, but vary k from 1 to 50. **

Similar to the digit recognition cross validation exercise, vary `k` from 1 to 50. With each `k`, create a new KNeighborsClassifier where n_neighbors = k. Then, use cross_val_score() with cv=5, and mean() the results of the 5 test partitions. Append this number onto a list.

Once you have this list, use pyplot.plot (or plt.plot if you have pyplot aliased), to plot out the trend.



--------------------

Hmm, scores seem quite low. What can we do to improve? Let's take a look at what the histograms of our features look like

In [ ]:
plt.figure(figsize=(20,20))
for idx, col in enumerate(X.columns):
    plt.subplot(2,7,idx+1)
    plt.hist( X[col])
    plt.xlabel(col)

Ah, all of the histograms are in different scales!! Pts are from 0 to 2500 or so, while BLKs might be from 0 to 200. This could definitely be messing up our distance function. i.e. For a prediction on a new player, it could just be choosing all the players with BLKs close to it, and ignoring PTS since there's such a large spread.

** In class drawing of scales **

In [ ]:
X.head()

In [ ]:
plt.figure(figsize=(20,20))
for idx, col in enumerate(X.columns):
    plt.subplot(2,7,idx+1)
    plt.hist( X[col] / X[col].std() )
    plt.xlabel(col)


Once we've transformed these points, looks much better

In [ ]:
X2 = X.copy()
X2 = X2.apply(lambda x: x / np.std(x),axis=1) #Remember to use axis=1, else it will divide by std of each row instead of each column

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X2, y, train_size=.8)  # Use 80% of the data for training
# Create our model based on 5 neighbors
model = neighbors.KNeighborsClassifier(n_neighbors=5)  # init kNN, with k=2
# Fit our model using training data
model.fit(X_train, y_train)
# Test/Score the model on training data
print model.score( X_train, y_train )
# Test/Score the model on testing data
print model.score( X_test, y_test )

from sklearn.cross_validation import cross_val_score
cv_scores = cross_val_score(model, X2, y, cv=5)  # use 10 different test sets
cv_scores.mean()

In [ ]:
cvArray = []
for k in range(1,50):
    model = neighbors.KNeighborsClassifier(n_neighbors=k)  # init KNN with k neighbors
    cv_scores = cross_val_score(model, X2, y, cv=10)  # use 5 different test sets
    cvArray.append(cv_scores.mean())

plt.plot(cvArray)